In [2]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup


### Census.gov API Call

In [41]:
response = None
url = 'https://api.census.gov/data/timeseries/poverty/saipe?get=SAEMHI_MOE,SAEMHI_PT,SAEMHI_UB90,SAEPOV0_17_LB90,SAEPOV0_17_MOE,SAEPOV0_17_PT,SAEPOV0_17_UB90,SAEPOV0_4_LB90,SAEPOV0_4_MOE,SAEPOV0_4_PT,SAEPOV0_4_UB90,SAEPOV5_17R_LB90,SAEPOV5_17R_MOE,SAEPOV5_17R_PT,SAEPOV5_17R_UB90,SAEPOVALL_LB90,SAEPOVALL_MOE,SAEPOVALL_PT,SAEPOVALL_UB90,SAEPOVRT0_17_LB90,SAEPOVRT0_17_MOE,SAEPOVRT0_17_PT,SAEPOVRT0_17_UB90,SAEPOVRT0_4_LB90,SAEPOVRT0_4_MOE,SAEPOVRT0_4_PT,SAEPOVRT0_4_UB90,SAEPOVRT5_17R_LB90,SAEPOVRT5_17R_MOE,SAEPOVRT5_17R_PT,SAEPOVRT5_17R_UB90,SAEPOVRTALL_LB90,SAEPOVRTALL_MOE,SAEPOVRTALL_PT,SAEPOVRTALL_UB90,SAEPOVU_0_17,SAEPOVU_0_4,SAEPOVU_5_17R,SAEPOVU_ALL,YEAR,STABREV&for=county:*&in=state:*'
poverty_response = requests.get(url)

Convert our api response to json

In [42]:
poverty_response_json = poverty_response.json()

load Poverty Data in DF

In [43]:
poverty_df = pd.DataFrame(poverty_response_json[1:], columns=poverty_response_json[0])
poverty_df


,SAEMHI_MOE,SAEMHI_PT,SAEMHI_UB90,SAEPOV0_17_LB90,SAEPOV0_17_MOE,SAEPOV0_17_PT,SAEPOV0_17_UB90,SAEPOV0_4_LB90,SAEPOV0_4_MOE,SAEPOV0_4_PT,...,SAEPOVRTALL_PT,SAEPOVRTALL_UB90,SAEPOVU_0_17,SAEPOVU_0_4,SAEPOVU_5_17R,SAEPOVU_ALL,YEAR,STABREV,state,county
0,2222,26898,29113,1665,406,2071,2477,None,None,None,...,14.3,17.1,None,None,None,None,1989,AL,01,001
1,2240,24043,26276,3872,967,4838,5805,None,None,None,...,13.2,15.8,None,None,None,None,1989,AL,01,003
2,2274,18673,20941,2257,570,2826,3396,None,None,None,...,26.2,31.3,None,None,None,None,1989,AL,01,005
3,2253,19604,21851,968,238,1206,1444,None,None,None,...,18.9,22.6,None,None,None,None,1989,AL,01,007
4,2248,24035,26276,1554,418,1972,2390,None,None,None,...,13.4,16.0,None,None,None,None,1989,AL,01,009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91108,7749,74677,82426,674,316,990,1306,None,None,None,...,9.2,10.9,10242,None,7889,40738,2021,WY,56,037
91109,12577,102709,115286,153,70,223,293,None,None,None,...,5.9,7.3,4067,None,3013,23406,2021,WY,56,039
91110,7956,70162,78118,426,209,635,844,None,None,None,...,9.4,11.6,5648,None,4408,20487,2021,WY,56,041
91111,6761,62176,68937,131,64,195,259,None,None,None,...,10.2,12.8,1562,None,1197,7575,2021,WY,56,043


In [44]:
poverty_df2 = poverty_df.rename(columns={"state": "State FIPS Code", "county": "County FIPS Code", 'STABREV': 'STATE' })

In [45]:
poverty_df2.head(20)

,SAEMHI_MOE,SAEMHI_PT,SAEMHI_UB90,SAEPOV0_17_LB90,SAEPOV0_17_MOE,SAEPOV0_17_PT,SAEPOV0_17_UB90,SAEPOV0_4_LB90,SAEPOV0_4_MOE,SAEPOV0_4_PT,...,SAEPOVRTALL_PT,SAEPOVRTALL_UB90,SAEPOVU_0_17,SAEPOVU_0_4,SAEPOVU_5_17R,SAEPOVU_ALL,YEAR,STATE,State FIPS Code,County FIPS Code
0,2222,26898,29113,1665,406,2071,2477,None,None,None,...,14.3,17.1,None,None,None,None,1989,AL,01,001
1,2240,24043,26276,3872,967,4838,5805,None,None,None,...,13.2,15.8,None,None,None,None,1989,AL,01,003
2,2274,18673,20941,2257,570,2826,3396,None,None,None,...,26.2,31.3,None,None,None,None,1989,AL,01,005
3,2253,19604,21851,968,238,1206,1444,None,None,None,...,18.9,22.6,None,None,None,None,1989,AL,01,007
4,2248,24035,26276,1554,418,1972,2390,None,None,None,...,13.4,16.0,None,None,None,None,1989,AL,01,009
5,2362,13273,15628,1147,289,1436,1725,None,None,None,...,34.1,40.7,None,None,None,None,1989,AL,01,011
6,2294,16683,18971,1977,490,2467,2957,None,None,None,...,26.3,31.4,None,None,None,None,1989,AL,01,013
7,2241,23688,25923,4472,1104,5576,6679,None,None,None,...,15.2,18.2,None,None,None,None,1989,AL,01,015
8,2248,21277,23518,1515,377,1892,2269,None,None,None,...,15.6,18.7,None,None,None,None,1989,AL,01,017
9,2262,21072,23327,633,160,793,953,None,None,None,...,14.2,16.9,None,None,None,None,1989,AL,01,019


Unused CSV

In [7]:
national2018_df = pd.read_excel("national_M2018_dl.xlsx")
national2019_df = pd.read_excel("national_M2019_dl.xlsx")
national2020_df = pd.read_excel("national_M2020_dl.xlsx")
national2021_df = pd.read_excel("national_M2021_dl.xlsx")
national2017_df = pd.read_excel("national_M2017_dl.xlsx")
state2018_df = pd.read_excel("state_M2018_dl.xlsx")
state2019_df = pd.read_excel("state_M2019_dl.xlsx")
state2020_df = pd.read_excel("state_M2020_dl.xlsx")
state2021_df = pd.read_excel("state_M2021_dl.xlsx")
state2017_df = pd.read_excel("state_M2017_dl.xlsx")


Add Year columns to data

In [8]:
national2017_df['Year'] = 2017
national2018_df['Year'] = 2018
national2019_df['Year'] = 2019
national2020_df['Year'] = 2020
national2021_df['Year'] = 2021
state2017_df['Year'] = 2017
state2018_df['Year'] = 2018
state2019_df['Year'] = 2019
state2020_df['Year'] = 2020
state2021_df['Year'] = 2021


Reusable function to change column names to upper

In [9]:
def upper_columns(data):
    columns = data
    myList = []
    for name in columns:
        myList.append(name.upper())
    data.columns = myList

In [10]:
df_list = ['national2017_df','national2018_df','national2019_df','national2020_df','national2021_df','state2017_df', 'state2018_df','state2019_df', 'state2020_df', 'state2021_df']

In [11]:
upper_columns(national2017_df)
upper_columns(national2018_df)
upper_columns(national2019_df)
upper_columns(national2020_df)
upper_columns(national2021_df)
upper_columns(state2017_df)
upper_columns(state2018_df)
upper_columns(state2019_df)
upper_columns(state2020_df)
upper_columns(state2021_df)

Rename columns that dont match

In [12]:
national2019_df = national2019_df.rename(columns={'O_GROUP': 'OCC_GROUP'})
national2020_df = national2020_df.rename(columns={'O_GROUP': 'OCC_GROUP'})
national2021_df = national2021_df.rename(columns={'O_GROUP': 'OCC_GROUP'})

Filter columns and set correct column order

In [31]:
national2017_ordered = national2017_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'EMP_PRSE', 'H_MEAN',
    'A_MEAN', 'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75',
    'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90',
    'YEAR']]
national2018_ordered = national2018_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'EMP_PRSE', 'H_MEAN',
    'A_MEAN', 'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75',
    'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90',
    'YEAR']]
national2019_ordered = national2019_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'EMP_PRSE', 'H_MEAN',
    'A_MEAN', 'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75',
    'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90',
    'YEAR']]
national2020_ordered = national2020_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'EMP_PRSE', 'H_MEAN',
    'A_MEAN', 'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75',
    'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90',
    'YEAR']]
national2021_ordered = national2021_df[[
    'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'EMP_PRSE', 'H_MEAN',
    'A_MEAN', 'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75',
    'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90',
    'YEAR']]

Concat all National data into one df

In [32]:
national_df = pd.concat([national2017_ordered, national2018_ordered, national2019_ordered, national2020_ordered, national2021_ordered], axis=0)
national_df.shape

(6822, 19)

In [34]:
national_df.isna().sum()

OCC_CODE     0
OCC_TITLE    0
OCC_GROUP    0
TOT_EMP      0
EMP_PRSE     0
H_MEAN       0
A_MEAN       0
MEAN_PRSE    0
H_PCT10      0
H_PCT25      0
H_MEDIAN     0
H_PCT75      0
H_PCT90      0
A_PCT10      0
A_PCT25      0
A_MEDIAN     0
A_PCT75      0
A_PCT90      0
YEAR         0
dtype: int64

In [35]:
national_df.head(20)

,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,YEAR
0,00-0000,All Occupations,total,142549250,0.1,24.34,50620,0.1,9.6,11.91,18.12,29.38,46.23,19970,24770,37690,61110,96150,2017
1,11-0000,Management Occupations,major,7280330,0.2,57.65,119910,0.1,23.19,33.6,49.32,71.83,#,48220,69880,102590,149410,#,2017
2,11-1000,Top Executives,minor,2473740,0.3,61.55,128020,0.2,20.74,31.74,49.58,78.72,#,43140,66030,103120,163740,#,2017
3,11-1010,Chief Executives,broad,210160,0.7,94.25,196050,0.4,32.74,54.55,88.11,#,#,68110,113470,183270,#,#,2017
4,11-1011,Chief Executives,detailed,210160,0.7,94.25,196050,0.4,32.74,54.55,88.11,#,#,68110,113470,183270,#,#,2017
5,11-1020,General and Operations Managers,broad,2212200,0.3,59.35,123460,0.2,21.4,31.53,48.27,75.28,#,44510,65590,100410,156580,#,2017
6,11-1021,General and Operations Managers,detailed,2212200,0.3,59.35,123460,0.2,21.4,31.53,48.27,75.28,#,44510,65590,100410,156580,#,2017
7,11-1030,Legislators,broad,51380,1.3,*,46350,1.0,*,*,*,*,*,17480,18860,25630,70130,97510,2017
8,11-1031,Legislators,detailed,51380,1.3,*,46350,1.0,*,*,*,*,*,17480,18860,25630,70130,97510,2017
9,11-2000,"Advertising, Marketing, Promotions, Public Rel...",minor,685780,0.5,66.66,138650,0.3,29.1,41.13,59.19,82.51,#,60530,85560,123100,171630,#,2017


Rename columns with inconsistent names

In [15]:
state2019_df = state2019_df.rename(columns={'AREA_TITLE': 'STATE'})
state2020_df = state2020_df.rename(columns={'AREA_TITLE': 'STATE'})
state2021_df = state2021_df.rename(columns={'AREA_TITLE': 'STATE'})

state2019_df = state2019_df.rename(columns={'O_GROUP': 'OCC_GROUP'})
state2020_df = state2020_df.rename(columns={'O_GROUP': 'OCC_GROUP'})
state2021_df = state2021_df.rename(columns={'O_GROUP': 'OCC_GROUP'})

state2017_df = state2017_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})
state2018_df = state2018_df.rename(columns={'LOC_Q': 'LOC_QUOTIENT'})


Filter columns and set correct column order

In [27]:
state2017_ordered = state2017_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
state2018_ordered = state2018_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
state2019_ordered = state2019_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
state2020_ordered = state2020_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]
state2021_ordered = state2021_df[[
    'AREA', 'STATE', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP',
    'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE',
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10',
    'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'YEAR']]

Concat all State data into one df

In [63]:
state_df = pd.concat([state2017_ordered, state2018_ordered, state2019_ordered, state2020_ordered, state2021_ordered], axis=0)
state_df.shape

(183936, 23)

In [64]:
states = {
    'Alaska': 'AK',
    'Alabama': 'AL',
    'Arkansas': 'AR',
    'Arizona': 'AZ',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'District of Columbia': 'DC',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Iowa': 'IA',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Massachusetts': 'MA',
    'Maryland': 'MD',
    'Maine': 'ME',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Missouri': 'MO',
    'Mississippi': 'MS',
    'Montana': 'MT',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Nebraska': 'NE',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'Nevada': 'NV',
    'New York': 'NY',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Virginia': 'VA',
    'Vermont': 'VT',
    'Washington': 'WA',
    'Wisconsin': 'WI',
    'West Virginia': 'WV',
    'Wyoming': 'WY'
}
state_df['STATE'] = state_df['STATE'].replace(states)

In [65]:
terr = ['Guam', 'Puerto Rico', 'Virgin Islands']
state_df = state_df[state_df['STATE'] != 'Guam'] 
state_df = state_df[state_df['STATE'] != 'Puerto Rico'] 
state_df = state_df[state_df['STATE'] != 'Virgin Islands']



In [66]:
state_df['STATE'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [29]:
state_df.isna().sum()

AREA            0
STATE           0
OCC_CODE        0
OCC_TITLE       0
OCC_GROUP       0
TOT_EMP         0
EMP_PRSE        0
JOBS_1000       0
LOC_QUOTIENT    0
H_MEAN          0
A_MEAN          0
MEAN_PRSE       0
H_PCT10         0
H_PCT25         0
H_MEDIAN        0
H_PCT75         0
H_PCT90         0
A_PCT10         0
A_PCT25         0
A_MEDIAN        0
A_PCT75         0
A_PCT90         0
YEAR            0
dtype: int64